<a href="https://colab.research.google.com/github/lapaniku/generative-sentiment/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
urllib.request.urlretrieve("https://gist.githubusercontent.com/lapaniku/90661c72db40684310a3033bfdcc2e78/raw/04170f67d729657d1c234ebaa6763b877b56116f/regen.json", "regen.json")

('regen.json', <http.client.HTTPMessage at 0x7a35c3016050>)

In [2]:

def translate_score_to_sentiment(label: int) -> str:
    return ["Strong Negative", "Mild Negative", "Neutral", "Mild Positive", "Strong Positive"][label]

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json

tokenizer = AutoTokenizer.from_pretrained("alapanik/blair-roberta-base-generative-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("alapanik/blair-roberta-base-generative-sentiment")

with open("regen.json", "r") as f:
  data = json.load(f)

sentences = [item["ticket"] for item in data]

inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
  outputs = model(**inputs)

predicted = torch.argmax(outputs.logits, dim=1)
gt = [item["output"] for item in data]
predicted_labels = [translate_score_to_sentiment(label) for label in predicted]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
precision_macro = precision_score(gt, predicted_labels, average='macro', zero_division=0)
recall_macro = recall_score(gt, predicted_labels, average='macro', zero_division=0)
precision_micro = precision_score(gt, predicted_labels, average='micro', zero_division=0)
recall_micro = recall_score(gt, predicted_labels, average='micro', zero_division=0)
conf_matrix = confusion_matrix(gt, predicted_labels).tolist()


In [10]:
precision_macro

0.8969367958893024

In [11]:

conf_matrix

[[284, 0, 8, 18, 0],
 [0, 97, 5, 0, 8],
 [32, 5, 102, 0, 0],
 [12, 0, 0, 127, 0],
 [0, 8, 0, 0, 302]]